### Imports

In [108]:
# Scinet modules
from scinet import *

# My custom modules
import scinet.ed_stokes as sto

#Other modules
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np

%load_ext tensorboard
import tensorflow as tf
import datetime, os


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Helper Functions (based on oscillator example)

In [4]:
def gen_input_stokes(Fr, St, t_observ, tt_predicted):
    in1 = np.array([sto.stokes_eqn(Fr, St, t_observ) for _ in tt_predicted])
    in2 = np.reshape(tt_predicted, (-1, 1))
    out = in2 #dummy filler
    return [in1, in2, out]

def stokes_representation_plot(net, Fr_range, St_range, t_observ, step_num=100, eval_time=1.5):
    Fr_vec = np.linspace(*Fr_range, num=step_num)
    St_vec = np.linspace(*St_range, num=step_num)
    
    FR, ST = np.meshgrid(Fr_vec, St_vec)
    
    eval_time = np.array([eval_time])
    out = np.array([net.run(gen_input_stokes(Fr, St, t_observ, eval_time), net.mu)[0] for Fr, St in zip(np.ravel(FR), np.ravel(ST))])
    
    fig = plt.figure(figsize=(net.latent_size*3.9, 2.1))
    for i in range(net.latent_size):
        zs = out[:, i]
        ax = fig.add_subplot('1{}{}'.format(net.latent_size, i + 1), projection='3d')
        Z = np.reshape(zs, FR.shape)
        surf = ax.plot_surface(FR, ST, Z, rstride=1, cstride=1, cmap=cm.inferno, linewidth=0)
        ax.set_xlabel(r'$Fr$')
        ax.set_ylabel(r'$St$')
        ax.set_zlabel('Latent activation {}'.format(i + 1))
        if (i==2):
            ax.set_zlim(-1,1) #Fix the scale for the third plot, where the activation is close to zero
        ax.set_zticks([-1,-0.5,0,0.5,1])
    fig.tight_layout()
    return fig

def plot_stokes_prediction(net_, Fr_, St_, t_observ, t_predict):    
    x_correct = sto.stokes_eqn(Fr, St, t_predict)
    x_predict = net.run(gen_input_stokes(Fr_, St_, t_observ, t_predict), net.output).ravel()
    fig = plt.figure(figsize=(7, 4))
    ax = fig.add_subplot(111)
    ax.plot(t_predict, x_correct, color=orange_color, label='True time evolution')
    ax.plot(t_predict, x_predict, '--', color=blue_color, label='Predicted time evolution')
    ax.set_xlabel(r'$t$ [$s$]')
    ax.set_ylabel(r'$x$ [$m$]')
    handles, labels = ax.get_legend_handles_labels()
    lgd=ax.legend(handles, labels,loc='upper left', bbox_to_anchor=(0.6, 1.3), shadow=True, ncol=1)
    fig.tight_layout()
    return fig

In [5]:
blue_color='#000cff'
orange_color='#ff7700'

### Input Variables

In [98]:
observation_size = 50
latent_size = 3
question_size = 1
answer_size = 1
dev_percent = 5
num_examples = 200000

encoder_layout = [500,100]
decoder_layout = [100,100]

t_sim = np.linspace(0,1,observation_size)
t_q = 2.0

### Data creation and loading

In [99]:
sto.stokes_data(num_examples, t_sample=t_sim, fileName='stokes_example');


In [100]:
td, vd, ts, vs, proj = dl.load(dev_percent, 'stokes_example')

### Create and train neural network

In [101]:
# Create network object
net = nn.Network(observation_size, latent_size, question_size, answer_size, encoder_num_units=encoder_layout, decoder_num_units=decoder_layout) 

In [102]:
# Print initial reconstruction loss (depends on initialization)
print(net.run(vd, net.recon_loss)) #default
print(net.run(vd, net.kl_loss))

3.6560032
16.279694


In [103]:
# Train
num_epochs = 2000
print_frequency = 0.2

check_epochs = int(print_frequency * num_epochs)
num_loops = int(num_epochs/check_epochs)

for i in range(num_loops):
    net.train(check_epochs, 512, 0.001, td, vd, beta_fun=(lambda x: 0.001) )
    # Check progress. It is recommended to use Tensorboard instead for this.
    print(net.run(vd, net.recon_loss)) #default
    print(net.run(vd, net.kl_loss))

  0%|          | 0/400 [00:00<?, ?it/s]

0.00068478734
5.0145135


  0%|          | 0/400 [00:00<?, ?it/s]

0.0002146625
4.921312


  0%|          | 0/400 [00:00<?, ?it/s]

0.00019341696
4.969851


  0%|          | 0/400 [00:00<?, ?it/s]

0.0003041797
4.9062986


  0%|          | 0/400 [00:00<?, ?it/s]

0.00017682654
4.9405212


In [104]:
# Check progress. It is recommended to use Tensorboard instead for this.
print(net.run(vd, net.recon_loss)) #default
print(net.run(vd, net.kl_loss))

0.00017682654
4.9405212


In [105]:
# Plot prediction
%matplotlib tk
Fr = 1.0
St = 3.0
t_predict = np.linspace(0, t_q, 250)
plot_stokes_prediction(net, Fr, St, t_sim, t_predict);

In [106]:
stokes_representation_plot(net, [-5.0,5.0], [0.1,5.0], t_sim, eval_time=1.5);

In [127]:
#Choose Network Name for saving (if desired)
date_str  = str(datetime.datetime.now().date())
name_str  = 'stokesNet_'
filename  = name_str + date_str
full_path = io.tf_save_path + filename

In [128]:
#Save Network
if os.path.isfile(full_path+'.pkl'):
    print("Filename already exists. Please choose another name.")
else:    
    net.save(filename)

Filename already exists. Please choose another name.
